<a href="https://colab.research.google.com/github/jhchang/Tensorflow_projects/blob/main/Vegetable_Image_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Need a compute power of 7.0+ to use mixed precision training
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-d911830c-ac5f-8311-f30c-9d2d010e18bc)


In [ ]:
import tensorflow as tf
print(tf.__version__)

2.9.2


In [ ]:
import numpy as np

In [ ]:
# Get helper functions file
import os 

if not os.path.exists("helper_functions.py"):
    !wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
else:
    print("[INFO] 'helper_functions.py' already exists, skipping download.")

[INFO] 'helper_functions.py' already exists, skipping download.


In [ ]:
from helper_functions import create_tensorboard_callback, plot_loss_curves, compare_historys, unzip_data, walk_through_dir

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
if os.path.exists('Vegetable Images/'):
    print('Directory already exists')
else:
    unzip_data('/content/drive/MyDrive/VeggieImageDataset.zip')

Directoy already exists


In [ ]:
train_dir = "Vegetable Images/train"
test_dir = "Vegetable Images/validation"

In [ ]:
walk_through_dir("Vegetable Images")

There are 3 directories and 0 images in 'Vegetable Images'.
There are 15 directories and 0 images in 'Vegetable Images/test'.
There are 0 directories and 200 images in 'Vegetable Images/test/Tomato'.
There are 0 directories and 200 images in 'Vegetable Images/test/Cabbage'.
There are 0 directories and 200 images in 'Vegetable Images/test/Bitter_Gourd'.
There are 0 directories and 200 images in 'Vegetable Images/test/Radish'.
There are 0 directories and 200 images in 'Vegetable Images/test/Bean'.
There are 0 directories and 200 images in 'Vegetable Images/test/Cauliflower'.
There are 0 directories and 200 images in 'Vegetable Images/test/Cucumber'.
There are 0 directories and 200 images in 'Vegetable Images/test/Capsicum'.
There are 0 directories and 200 images in 'Vegetable Images/test/Carrot'.
There are 0 directories and 200 images in 'Vegetable Images/test/Papaya'.
There are 0 directories and 200 images in 'Vegetable Images/test/Bottle_Gourd'.
There are 0 directories and 200 images i

In [ ]:
IMG_SIZE = (224, 224)
train_data = tf.keras.preprocessing.image_dataset_from_directory(train_dir,
                                                                 label_mode="categorical",
                                                                 batch_size=None,
                                                                 image_size=IMG_SIZE,
                                                                 shuffle=False)
                                                                                
test_data = tf.keras.preprocessing.image_dataset_from_directory(test_dir,
                                                                label_mode="categorical",
                                                                image_size=IMG_SIZE,
                                                                batch_size=None,
                                                                shuffle=False) # don't shuffle test data

Found 15000 files belonging to 15 classes.
Found 3000 files belonging to 15 classes.


In [ ]:
class_names = train_data.class_names
print(class_names)

['Bean', 'Bitter_Gourd', 'Bottle_Gourd', 'Brinjal', 'Broccoli', 'Cabbage', 'Capsicum', 'Carrot', 'Cauliflower', 'Cucumber', 'Papaya', 'Potato', 'Pumpkin', 'Radish', 'Tomato']


In [ ]:
for images, labels in train_data.take(1):
  print(images, labels)

tf.Tensor(
[[[184. 181. 188.]
  [185. 182. 189.]
  [185. 182. 189.]
  ...
  [192. 192. 190.]
  [192. 193. 188.]
  [192. 193. 188.]]

 [[184. 181. 188.]
  [185. 182. 189.]
  [185. 182. 189.]
  ...
  [192. 192. 190.]
  [192. 193. 188.]
  [192. 193. 188.]]

 [[184. 181. 188.]
  [185. 182. 189.]
  [185. 182. 189.]
  ...
  [191. 191. 189.]
  [191. 192. 187.]
  [191. 192. 187.]]

 ...

 [[169. 164. 161.]
  [169. 164. 161.]
  [169. 164. 161.]
  ...
  [175. 174. 172.]
  [175. 174. 172.]
  [175. 174. 172.]]

 [[171. 163. 161.]
  [171. 163. 161.]
  [171. 163. 161.]
  ...
  [174. 173. 169.]
  [174. 173. 169.]
  [174. 173. 169.]]

 [[170. 162. 160.]
  [170. 162. 160.]
  [171. 163. 161.]
  ...
  [174. 173. 169.]
  [174. 173. 169.]
  [174. 173. 169.]]], shape=(224, 224, 3), dtype=float32) tf.Tensor([1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], shape=(15,), dtype=float32)


In [ ]:
# Output info about our training sample
for image, label in train_data.take(1):
  print(f"""
  Image shape: {image.shape}
  Image dtype: {image.dtype}
  Target class from Food101 (tensor form): {np.array(label).argmax()}
  Class name (str form): {class_names[np.array(label).argmax()]}
        """)


  Image shape: (224, 224, 3)
  Image dtype: <dtype: 'float32'>
  Target class from Food101 (tensor form): 0
  Class name (str form): Bean
        


In [ ]:
def preprocess_img(image, label, img_shape=224):
    """
    Converts image datatype from 'uint8' -> 'float32' and reshapes image to
    [img_shape, img_shape, color_channels]
    """
    image = tf.image.resize(image, [img_shape, img_shape]) # reshape to img_shape
    return tf.cast(image, tf.float32), tf.cast(label, tf.int64) # return (float32_image, label) tuple

In [ ]:
# Map preprocessing function to training data (and paralellize)
train_data = train_data.map(map_func=preprocess_img, num_parallel_calls=tf.data.AUTOTUNE)
# Shuffle train_data and turn it into batches and prefetch it (load it faster)
train_data = train_data.shuffle(buffer_size=1000).batch(batch_size=32).prefetch(buffer_size=tf.data.AUTOTUNE)

# Map prepreprocessing function to test data
test_data = test_data.map(preprocess_img, num_parallel_calls=tf.data.AUTOTUNE)
# Turn test data into batches (don't need to shuffle)
test_data = test_data.batch(32).prefetch(tf.data.AUTOTUNE)

In [ ]:
train_data, test_data

(<PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 15), dtype=tf.int64, name=None))>,
 <PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 15), dtype=tf.int64, name=None))>)

In [ ]:
'''
Callbacks for transfer learning
'''
checkpoint_path = "model_checkpoints/cp.ckpt" # saving weights requires ".ckpt" extension
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                      monitor="val_accuracy", # save the model weights with best validation accuracy
                                                      save_best_only=True, # only save the best weights
                                                      save_weights_only=True, # only save model weights (not whole model)
                                                      verbose=1) # print out whether or not model is being saved 

In [ ]:
from tensorflow.keras import layers

# Create base model
input_shape = (224, 224, 3)
base_model = tf.keras.applications.EfficientNetB0(include_top=False)
base_model.trainable = False # freeze base model layers

# Create Functional model 
inputs = layers.Input(shape=input_shape, name="input_layer")
# Note: EfficientNetBX models have rescaling built-in but if your model didn't you could have a layer like below
# x = layers.Rescaling(1./255)(x)
x = base_model(inputs, training=False) # set base_model to inference mode only
x = layers.GlobalAveragePooling2D(name="pooling_layer")(x)
outputs = layers.Dense(len(class_names), activation="softmax")(x) # want one output neuron per class 
model = tf.keras.Model(inputs, outputs)

# Compile the model
model.compile(loss="categorical_crossentropy", # Use sparse_categorical_crossentropy when labels are *not* one-hot
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 efficientnetb0 (Functional)  (None, None, None, 1280)  4049571  
                                                                 
 pooling_layer (GlobalAverag  (None, 1280)             0         
 ePooling2D)                                                     
                                                                 
 dense (Dense)               (None, 15)                19215     
                                                                 
Total params: 4,068,786
Trainable params: 19,215
Non-trainable params: 4,049,571
_________________________________________________________________


In [ ]:
# Turn off all warnings except for errors
tf.get_logger().setLevel('ERROR')

# Fit the model with callbacks
history_efficientnetb0_veg_feature_extract = model.fit(train_data, 
                                                     epochs=3,
                                                     steps_per_epoch=len(train_data),
                                                     validation_data=test_data,
                                                     validation_steps=int(0.15 * len(test_data)),
                                                     callbacks=[create_tensorboard_callback("training_logs", 
                                                                                            "efficientnetb0_veg_feature_extract"),
                                                                model_checkpoint])

Saving TensorBoard log files to: training_logs/efficientnetb0_veg_feature_extract/20230111-003902
Epoch 1/3
469/469 [==============================] - ETA: 0s - loss: 0.4062 - accuracy: 0.8975
Epoch 1: val_accuracy improved from -inf to 0.39062, saving model to model_checkpoints/cp.ckpt
469/469 [==============================] - 45s 75ms/step - loss: 0.4062 - accuracy: 0.8975 - val_loss: 1.4626 - val_accuracy: 0.3906
Epoch 2/3
469/469 [==============================] - ETA: 0s - loss: 0.0728 - accuracy: 0.9799
Epoch 2: val_accuracy improved from 0.39062 to 0.95089, saving model to model_checkpoints/cp.ckpt
469/469 [==============================] - 34s 71ms/step - loss: 0.0728 - accuracy: 0.9799 - val_loss: 0.1523 - val_accuracy: 0.9509
Epoch 3/3
469/469 [==============================] - ETA: 0s - loss: 0.0286 - accuracy: 0.9946
Epoch 3: val_accuracy improved from 0.95089 to 0.97768, saving model to model_checkpoints/cp.ckpt
469/469 [==============================] - 34s 71ms/step - l

In [ ]:
# Evaluate model (unsaved version) on whole test dataset
results_feature_extract_model = model.evaluate(test_data)
results_feature_extract_model

94/94 [==============================] - 6s 63ms/step - loss: 0.0806 - accuracy: 0.9687


[0.0805649533867836, 0.968666672706604]

In [ ]:
loaded_saved_model = tf.keras.models.clone_model(model)
loaded_saved_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 efficientnetb0 (Functional)  (None, None, None, 1280)  4049571  
                                                                 
 pooling_layer (GlobalAverag  (None, 1280)             0         
 ePooling2D)                                                     
                                                                 
 dense (Dense)               (None, 15)                19215     
                                                                 
Total params: 4,068,786
Trainable params: 19,215
Non-trainable params: 4,049,571
_________________________________________________________________


In [ ]:
checkpoint_path

'model_checkpoints/cp.ckpt'

In [ ]:
loaded_saved_model.load_weights(checkpoint_path)

In [ ]:
loaded_saved_model.compile(loss="categorical_crossentropy",
                           optimizer=tf.keras.optimizers.Adam(),
                           metrics=["accuracy"])

In [ ]:
# Evalaute cloned model with loaded weights (should be same score as trained model)
results_loaded_save_model = loaded_saved_model.evaluate(test_data)

94/94 [==============================] - 9s 62ms/step - loss: 0.0835 - accuracy: 0.9777


In [ ]:
'''
# Loaded checkpoint weights should return very similar results to checkpoint weights prior to saving
import numpy as np
assert np.isclose(results_feature_extract_model, results_cloned_model_with_loaded_weights).all(), "Loaded weights results are not close to original model."  # check if all elements in array are close
'''

'\n# Loaded checkpoint weights should return very similar results to checkpoint weights prior to saving\nimport numpy as np\nassert np.isclose(results_feature_extract_model, results_cloned_model_with_loaded_weights).all(), "Loaded weights results are not close to original model."  # check if all elements in array are close\n'

In [ ]:
'''
Callbacks for fine tuning
'''


# Create ModelCheckpoint callback to save best model during fine-tuning
checkpoint_path = "fine_tune_checkpoints/"
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                      save_best_only=True,
                                                      monitor="val_loss")

# Setup EarlyStopping callback to stop training if model's val_loss doesn't improve for 3 epochs
early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_loss", # watch the val loss metric
                                                  patience=3) # if val loss decreases for 3 epochs in a row, stop training

# Creating learning rate reduction callback
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss",  
                                                 factor=0.2, # multiply the learning rate by 0.2 (reduce by 5x)
                                                 patience=2,
                                                 verbose=1, # print out when learning rate goes down 
                                                 min_lr=1e-7)


In [ ]:
'''
TODO: set layers to be trainable
'''

'\nTODO: set layers to be trainable\n'

In [ ]:
# Check the layers in the base model and see what dtype policy they're using
for layer in loaded_saved_model.layers[1].layers[:20]: # check only the first 20 layers to save output space
    print(layer.name, layer.trainable, layer.dtype, layer.dtype_policy)

input_1 True float32 <Policy "float32">
rescaling False float32 <Policy "float32">
normalization False float32 <Policy "float32">
tf.math.truediv False float32 <Policy "float32">
stem_conv_pad False float32 <Policy "float32">
stem_conv False float32 <Policy "float32">
stem_bn False float32 <Policy "float32">
stem_activation False float32 <Policy "float32">
block1a_dwconv False float32 <Policy "float32">
block1a_bn False float32 <Policy "float32">
block1a_activation False float32 <Policy "float32">
block1a_se_squeeze False float32 <Policy "float32">
block1a_se_reshape False float32 <Policy "float32">
block1a_se_reduce False float32 <Policy "float32">
block1a_se_expand False float32 <Policy "float32">
block1a_se_excite False float32 <Policy "float32">
block1a_project_conv False float32 <Policy "float32">
block1a_project_bn False float32 <Policy "float32">
block2a_expand_conv False float32 <Policy "float32">
block2a_expand_bn False float32 <Policy "float32">


In [ ]:
for layer in loaded_saved_model.layers:
    layer.trainable = True # set all layers to trainable
    print(layer.name, layer.trainable, layer.dtype, layer.dtype_policy) # make sure loaded model is using mixed precision dtype_policy ("mixed_float16")

input_layer True float32 <Policy "float32">
efficientnetb0 True float32 <Policy "float32">
pooling_layer True float32 <Policy "float32">
dense True float32 <Policy "float32">


In [ ]:
# Check the layers in the base model and see what dtype policy they're using
for layer in loaded_saved_model.layers[1].layers[:20]:
    print(layer.name, layer.trainable, layer.dtype, layer.dtype_policy)

input_1 True float32 <Policy "float32">
rescaling True float32 <Policy "float32">
normalization True float32 <Policy "float32">
tf.math.truediv True float32 <Policy "float32">
stem_conv_pad True float32 <Policy "float32">
stem_conv True float32 <Policy "float32">
stem_bn True float32 <Policy "float32">
stem_activation True float32 <Policy "float32">
block1a_dwconv True float32 <Policy "float32">
block1a_bn True float32 <Policy "float32">
block1a_activation True float32 <Policy "float32">
block1a_se_squeeze True float32 <Policy "float32">
block1a_se_reshape True float32 <Policy "float32">
block1a_se_reduce True float32 <Policy "float32">
block1a_se_expand True float32 <Policy "float32">
block1a_se_excite True float32 <Policy "float32">
block1a_project_conv True float32 <Policy "float32">
block1a_project_bn True float32 <Policy "float32">
block2a_expand_conv True float32 <Policy "float32">
block2a_expand_bn True float32 <Policy "float32">


In [ ]:
# Refreeze every layer except for the last 5
for layer in base_model.layers[:-5]:
    layer.trainable = False

In [ ]:
# Check the layers in the base model and see what dtype policy they're using
for layer in loaded_saved_model.layers[1].layers[:-10]:
    print(layer.name, layer.trainable, layer.dtype, layer.dtype_policy)

input_1 True float32 <Policy "float32">
rescaling True float32 <Policy "float32">
normalization True float32 <Policy "float32">
tf.math.truediv True float32 <Policy "float32">
stem_conv_pad True float32 <Policy "float32">
stem_conv True float32 <Policy "float32">
stem_bn True float32 <Policy "float32">
stem_activation True float32 <Policy "float32">
block1a_dwconv True float32 <Policy "float32">
block1a_bn True float32 <Policy "float32">
block1a_activation True float32 <Policy "float32">
block1a_se_squeeze True float32 <Policy "float32">
block1a_se_reshape True float32 <Policy "float32">
block1a_se_reduce True float32 <Policy "float32">
block1a_se_expand True float32 <Policy "float32">
block1a_se_excite True float32 <Policy "float32">
block1a_project_conv True float32 <Policy "float32">
block1a_project_bn True float32 <Policy "float32">
block2a_expand_conv True float32 <Policy "float32">
block2a_expand_bn True float32 <Policy "float32">
block2a_expand_activation True float32 <Policy "f

In [ ]:
# Compile the model
loaded_saved_model.compile(loss="categorical_crossentropy", # sparse_categorical_crossentropy for labels that are *not* one-hot
                           optimizer=tf.keras.optimizers.Adam(0.0001), # 10x lower learning rate than the default
                           metrics=["accuracy"])

In [ ]:
# Start to fine-tune (all layers)
history_veg_fine_tune = loaded_saved_model.fit(train_data,
                                                        epochs=10, # fine-tune for a maximum of 100 epochs
                                                        steps_per_epoch=len(train_data),
                                                        validation_data=test_data,
                                                        validation_steps=int(0.15 * len(test_data)), # validation during training on 15% of test data
                                                        callbacks=[create_tensorboard_callback("training_logs", "efficientb0_veg_fine_tuning"), # track the model training logs
                                                                   model_checkpoint, # save only the best model during training
                                                                   early_stopping, # stop model after X epochs of no improvements
                                                                   reduce_lr]) # reduce the learning rate after X epochs of no improvements

Saving TensorBoard log files to: training_logs/efficientb0_veg_fine_tuning/20230111-004131
Epoch 1/100
469/469 [==============================] - ETA: 0s - loss: 0.0167 - accuracy: 0.9945

469/469 [==============================] - 217s 431ms/step - loss: 0.0167 - accuracy: 0.9945 - val_loss: 0.1251 - val_accuracy: 0.9576 - lr: 1.0000e-04
Epoch 2/100
469/469 [==============================] - ETA: 0s - loss: 0.0060 - accuracy: 0.9983

469/469 [==============================] - 204s 430ms/step - loss: 0.0060 - accuracy: 0.9983 - val_loss: 0.0715 - val_accuracy: 0.9688 - lr: 1.0000e-04
Epoch 3/100
469/469 [==============================] - 160s 337ms/step - loss: 0.0034 - accuracy: 0.9989 - val_loss: 0.0720 - val_accuracy: 0.9732 - lr: 1.0000e-04
Epoch 4/100
469/469 [==============================] - ETA: 0s - loss: 0.0109 - accuracy: 0.9971

469/469 [==============================] - 203s 429ms/step - loss: 0.0109 - accuracy: 0.9971 - val_loss: 0.0249 - val_accuracy: 0.9933 - lr: 1.0000e-04
Epoch 5/100
469/469 [==============================] - 161s 339ms/step - loss: 0.0042 - accuracy: 0.9989 - val_loss: 0.5334 - val_accuracy: 0.8170 - lr: 1.0000e-04
Epoch 6/100
469/469 [==============================] - ETA: 0s - loss: 0.0147 - accuracy: 0.9967
Epoch 6: ReduceLROnPlateau reducing learning rate to 1.9999999494757503e-05.
469/469 [==============================] - 160s 336ms/step - loss: 0.0147 - accuracy: 0.9967 - val_loss: 1.0636 - val_accuracy: 0.7054 - lr: 1.0000e-04
Epoch 7/100
469/469 [==============================] - ETA: 0s - loss: 0.0217 - accuracy: 0.9939

469/469 [==============================] - 203s 429ms/step - loss: 0.0217 - accuracy: 0.9939 - val_loss: 0.0016 - val_accuracy: 1.0000 - lr: 2.0000e-05
Epoch 8/100
469/469 [==============================] - 162s 341ms/step - loss: 6.6772e-04 - accuracy: 1.0000 - val_loss: 0.0030 - val_accuracy: 0.9978 - lr: 2.0000e-05
Epoch 9/100
469/469 [==============================] - ETA: 0s - loss: 2.5823e-04 - accuracy: 1.0000

469/469 [==============================] - 204s 430ms/step - loss: 2.5823e-04 - accuracy: 1.0000 - val_loss: 5.7531e-04 - val_accuracy: 1.0000 - lr: 2.0000e-05
Epoch 10/100
469/469 [==============================] - ETA: 0s - loss: 1.1596e-04 - accuracy: 1.0000

469/469 [==============================] - 204s 431ms/step - loss: 1.1596e-04 - accuracy: 1.0000 - val_loss: 5.0054e-04 - val_accuracy: 1.0000 - lr: 2.0000e-05
Epoch 11/100
469/469 [==============================] - ETA: 0s - loss: 7.0573e-05 - accuracy: 1.0000

469/469 [==============================] - 202s 428ms/step - loss: 7.0573e-05 - accuracy: 1.0000 - val_loss: 4.1141e-04 - val_accuracy: 1.0000 - lr: 2.0000e-05
Epoch 12/100
469/469 [==============================] - ETA: 0s - loss: 5.0998e-05 - accuracy: 1.0000

KeyboardInterrupt: ignored